In [ ]:
# Keysight Libraries for Digitizer & AWG
import sys
sys.path.append('C:\Program Files (x86)\Keysight\SD1\Libraries\Python')
import keysightSD1

# Visa Library for MW Sources
import visa
visaRM = visa.ResourceManager()
print(visaRM.list_resources())

# Useful Libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import time
import csv

# stuff for nidaq thing
import nidaqmx
from nidaqmx.constants import AcquisitionType as AcqType
from nidaqmx.constants import Signal
from nidaqmx.constants import DigitalWidthUnits as TimeUnit
system = nidaqmx.system.System.local()
device = system.devices['Dev3']

In [ ]:
# Digitizer Module Constants
PRODUCT = ""
CHASSIS = 1
SLOT = 3


In [ ]:
# MW Source Parameters
P_RF = -30           # E8267D power level dBm
f_start = 1e6        # E8267D frequency (Hz)
f_end= 10e6          # end freq (Hz)
f_step= 0.05e6       # increment amount (Hz)
t_point_acqu = 0.01  # acquisition time for one data point
DWELL=1e-3

# Digitizer Parameters
r_s = 500e6                          # M3102A sampling rate
n_points = 100000                   # number of points per cycle (number of measured points after each trigger)
n_cycles = 1                         # number of cycles (number of trigger signals)
n_total = n_points * n_cycles        # total number of measured points over all cycles
t_total = n_total / r_s              # total time over which data points are measured


In [ ]:
# Create and Open MW source Modules
with visaRM.open_resource('GPIB0::19::INSTR') as E8267D:
    # Set RF Power
    P_E8267D_str = 'SOUR:POW '+str(P_RF)+'dBm; *STB?'
    E8267D.query(P_E8267D_str) 
    
    f_E8267D_str = 'FREQ:MODE LIST; *STB?'
    E8267D.query(f_E8267D_str)
    
    f_E8267D_str = 'LIST:TYPE STEP; *STB?'
    E8267D.query(f_E8267D_str)
    
    #Set RF Freq start, stop and increment
    f_E8267D_str = 'FREQ:STAR '+str(f_start)+'Hz; *STB?'
    E8267D.query(f_E8267D_str)
    f_E8267D_str = 'FREQ:STOP '+str(f_end)+'Hz; *STB?'   
    E8267D.query(f_E8267D_str)
    f_E8267D_str = 'SWE:POIN '+str(10)+'; *STB?'
    E8267D.query(f_E8267D_str)

    #set frequency step dwell time (linear)
    f_E8267D_str = 'SWE:DWEL '+ str(DWELL)+'s; *STB?'
    E8267D.query(f_E8267D_str)
    
    f_E8267D_str = 'OUTP:STAT ON; *STB?'
    E8267D.query(f_E8267D_str)
    
    
    # TRIG:SOUR without :LIST: triggers the whole sweep at once
    f_E8267D_str = 'TRIG:SOUR KEY; *STB?'
    E8267D.query(f_E8267D_str)
    
    f_E8267D_str= 'TRIG:SLOP POS; *STB?'
    E8267D.query(f_E8267D_str)

#     acqu_task = nidaqmx.Task()
#     acqu_task.ai_channels.add_ai_voltage_chan("Dev3/ai0",min_val=-1,max_val=1)
#     acqu_task.export_signals.start_trig_output_term = 'pfi0'
#     acqu_task.start()
    
    # Create and Open Digitizer Module
#     M3102A = keysightSD1.SD_AIN()
#     M3102A_ID = M3102A.openWithSlot(PRODUCT, CHASSIS, SLOT)

#     if M3102A_ID < 0:
#         print("Module open error:", M3102A_ID)
#     else:
#         print("Module opened:", M3102A_ID)
#         print("Module name:", M3102A.getProductName())
#         print("slot:", M3102A.getSlot())
#         print("Chassis:", M3102A.getChassis())
#         print()
        
#         trig_delay = 0
#         ch = 4                  # channel number
#         M3102A.DAQconfig(ch, n_points, n_cycles, trig_delay, keysightSD1.SD_TriggerModes.EXTTRIG)
#         M3102A.DAQdigitalTriggerConfig(ch, keysightSD1.SD_TriggerExternalSources.TRIGGER_PXI3,
#         keysightSD1.SD_TriggerBehaviors.TRIGGER_RISE)
#         M3102A.DAQstart(ch)
#         PXI3 = 3
        
#         M3102A.triggerIOconfig(0)


#         M3102A.triggerIOwrite(0,syncMode = 1)
#         M3102A.PXItriggerWrite(PXI3, 1);
#         M3102A.triggerIOwrite(1, syncMode = 1)
#         M3102A.PXItriggerWrite(PXI3, 0);
#         M3102A.triggerIOwrite(0,syncMode = 1)
#         M3102A.PXItriggerWrite(PXI3, 1);

        
#         # READ DATA
#         TIMEOUT = 1
#         dataRead = M3102A.DAQread(ch, n_total, TIMEOUT) # dataRead is in units of uV by default (check??)
# #         print(dataRead)

#         M3102A.DAQstop(ch)
#         # exiting...  
#         M3102A.close()
#         print()
#         print("AIN closed")
        
    # Close RF Output
    E8267D.query('OUTP OFF; *STB?')
    

In [ ]:
print(len(dataRead))

In [ ]:
t = np.linspace(0,t_total,len(dataRead))

plt.figure(num=None, figsize=(10, 6), dpi=80, facecolor='w', edgecolor='k')
plt.plot(t[0:1000]*1e6 , dataRead[0:1000]*1e-3)
# plt.xlim(100,110)
plt.xlabel('t [us]',fontsize=20)
plt.ylabel('V [mV]',fontsize=20)
plt.grid(True)